To open this notebook in Google Colab and start coding, click on the Colab icon below.

<table style="border:2px solid orange" align="left">
  <td style="border:2px solid orange ">
    <a target="_blank" href="https://colab.research.google.com/github/ChristinaVS95/workshop_python_git/blob/main/notebooks/4_Statistics_and_Plotting.ipynb?nocache=20250606"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
 </table>

# Introduction to Statistics and Plotting in Python

**Source: https://peerherholz.github.io/workshop_weizmann/prerequisites/intro_statistics.html**

In this section, we will cover how you can use Python to do some statistics and visualize results. There are many packages to do so, but we will focus on:

- **pandas**
- **pingouin**
- **seaborn and matplotlib**

# Load packages

In [1]:
!pip install pingouin

import numpy as np
import pandas as pd
import pingouin as pg

import matplotlib.pyplot as plt
import seaborn as sns

# Load fictional dataset

Example fictional dataset: It contains 
- **subject**
- **group**
- **sex**
- **age**
- **Emo** (values of a fictional emotion regulation score)
- **Exp** (values of a fictional expectation score)
- **Symp** (values of a fictional depression score)
- **Symp_FU** (depression score at follow-up)
- **time** (morning vs. evening assessment)

**Run the code below to load the dataset:**

In [ ]:
data = pd.read_csv('../data/dataset_1.csv')
data.head()

In [2]:
url = 'https://raw.githubusercontent.com/ChristinaVS95/workshop_python_git/main/data/dataset_1.csv'
data = pd.read_csv(url)

<center><img src="https://upload.wikimedia.org/wikipedia/commons/thumb/e/ed/Pandas_logo.svg/512px-Pandas_logo.svg.png" width="45%"></center>


# The pandas data-frame

### Manipulating data

`data` is a `pandas.DataFrame`, that resembles R's dataframe:

In [ ]:
data.shape    # 200 rows and 9 columns

In [ ]:
data.columns  # It has columns

In [ ]:
print(data['group'])  # Columns can be addressed by name

In [ ]:
# Simpler selector
data[data['group'] == 'HC']['Emo'].mean()

**Note:** For a quick view on a large dataframe, use its `describe` `pandas.DataFrame.describe`.

In [ ]:
data_unique = data.drop_duplicates(subset='subject')
data_unique.describe()

In [ ]:
# Frequency count for a given column
data_unique['sex'].value_counts()

### The split-apply-combine pattern
* A very common data processing strategy is to...
    * Split the dataset into groups
    * Apply some operation(s) to each group
    * (Optionally) combine back into one dataset

Pandas provides powerful and fast tools for this. For example the `groupby` function.

**groupby**: splitting a dataframe on values of categorical variables:

In [ ]:
groupby_group = data.groupby('group')
for group, value in groupby_group['Emo']:
     print((group, value.mean()))

`groupby_group` is a powerful object that exposes many operations on the resulting group of `dataframes`:

In [ ]:
groupby_group[['Emo', 'Exp', 'Symp', 'age']].mean()

<img src="https://raw.githubusercontent.com/raphaelvallat/pingouin/master/docs/pictures/logo_pingouin.png" width="50%">


### _Pingouin is an open-source statistical package written in Python 3 and based mostly on Pandas and NumPy._




- ANOVAs: one- and two-ways, repeated measures, mixed, ancova
- Post-hocs tests and pairwise comparisons
- Robust correlations
- Partial correlation, repeated measures correlation and intraclass correlation
- Linear/logistic regression and mediation analysis
- Bayesian T-test and Pearson correlation
- Tests for sphericity, normality and homoscedasticity
- Effect sizes and power analysis
- Parametric/bootstrapped confidence intervals around an effect size or a correlation coefficient
- Circular statistics
- Plotting: Bland-Altman plot, Q-Q plot, etc...

**Pingouin is designed for users who want simple yet exhaustive statistical functions.**

In [ ]:
import pingouin as pg

## Correlations

In [ ]:
pearson_correlation = pg.corr(data['Emo'], data['Symp'])
display(pearson_correlation)
cor_coeeficient = pearson_correlation['r']

#### Test summary

- 'n' : Sample size (after NaN removal)
- 'r' : Correlation coefficient
- 'CI95' : [95% parametric confidence intervals](https://en.wikipedia.org/wiki/Confidence_interval)
- 'p-val' : one or two tailed p-value
- 'BF10' : Bayes Factor of the alternative hypothesis (Pearson only)
- 'power' : achieved power of the test (= 1 - type II error)

## Parametric tests

## 1-sample t-test: testing the value of a population mean

Tests if the population mean of data is likely to be equal to a given value, e.g., zero.

In [ ]:
pg.ttest(data['Symp'],0)

## 2-sample t-test: testing for difference across populations

We have seen above that the mean `Symp` in the `MDD` and `HC` populations
were different. To test if this is significant, we do a 2-sample t-test:

In [ ]:
MDD_Symp = data[data['group'] == 'MDD']['Symp']
HC_Symp = data[data['group'] == 'HC']['Symp']

In [ ]:
pg.ttest(MDD_Symp, HC_Symp)

## One-way analysis of variance (ANOVA)

Can we explain the symptom score by group membership?

In [ ]:
aov = pg.anova(dv='Symp', between='group', data=data, detailed=True)
pg.print_table(aov)

#### Interpretation:
The F-value of 187 with a highly significant p-value (< 0.001) indicates there is a very strong difference between the groups.

Partial Eta Squared (np2) = 0.65:
This represents a very large effect size — 65% of the variance in the dependent variable (Symp) is explained by group membership.

## Mixed analysis of variance (ANOVA)

Can we explain the symptom score by the combination of group membership and time of measurement?

In [ ]:
aov = pg.mixed_anova(data=data, dv='Symp', between='group', within='time',
                     subject='subject', correction=False, effsize="np2")
pg.print_table(aov)

#### Interpretation:

**Main effect of group:**
- The effect of the between-subjects factor group is highly significant, with a very large effect size (partial eta squared = 0.659).
- This means there is a strong difference in the dependent variable Symp between the groups (e.g., MDD vs. HC).

**Main effect of time:**
- The within-subject factor time (morning vs. evening) is also highly significant, with an even larger effect size (partial eta squared = 0.804).
- This indicates that, across all subjects, symptom scores significantly differ between morning and evening. Since you said Symp is higher in the evening, this confirms that Symp changes significantly over time.

**Interaction effect (group × time):**
- The interaction between group and time is not significant, and the effect size is very small (partial eta squared = 0.005).
- This suggests that the difference in symptom scores from morning to evening does not differ significantly between groups — both groups show a similar pattern of change over time.

## Post-hoc tests

In [ ]:
# FDR-corrected post hocs with Hedges'g effect size
posthoc = pg.pairwise_tests(data=data, dv='Symp', between='group', within='time',
                            subject='subject', parametric=True, padjust='fdr_bh', effsize='hedges')

df_posthoc = pd.DataFrame(posthoc)
df_posthoc

#### Interpretation

**Time Effect (evening vs. morning)**
- There is a highly significant difference between evening and morning symptom scores (p < 1e-35).
- The large t-value (20.09) indicates a strong difference.
- Hedges' g of 0.30 indicates a small to moderate effect size.
- This means symptoms tend to be higher in the evening compared to the morning.

**Group Effect (healthy controls (hc) vs. patients with MDD)**
- The difference between groups is also highly significant (p < 1e-24).
- The negative t-value (-13.78) suggests that the MDD group has higher symptom scores than healthy controls.
- Hedges' g around -2.73 represents a very large effect size.

**Interaction Time * Group (evening, hc vs. mdd)**
- Significant difference between groups in the evening (FDR-corrected p < 3e-24).
- Hedges' g near -2.7 indicates a very large effect size.
- This means that the groups differ substantially in symptom scores during the evening.

**Interaction Time * Group (morning, hc vs. mdd)**
- Significant difference between groups in the morning as well (FDR-corrected p < 3e-24).
- Again, a very large effect size (Hedges' g about -2.73).
- The groups also differ strongly in the morning symptom scores.

<center><img src="https://raw.githubusercontent.com/mwaskom/seaborn/master/doc/_static/logo-wide-lightbg.png" width="50%"></center>

# `seaborn` - use visualization for statistical exploration

Seaborn combines simple statistical fits with plotting on pandas dataframes.

Graph galleries: \
https://seaborn.pydata.org/examples/index.html \
https://www.python-graph-gallery.com/ \
https://plotly.com/python/

In [ ]:
import seaborn as sns

import warnings
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

## Plotting with categorical data

https://seaborn.pydata.org/generated/seaborn.catplot.html

In [ ]:
sns.set_style("whitegrid")                   # set the style of the plot (e.g., whitegrid, darkgrid, white, dark)
sns.set_context("paper", font_scale = 1.75)  # set the context of the plot (e.g., paper, notebook, talk, poster)

plot = sns.catplot(data   = data,     # dataset that is used for plotting
                   x      = 'group',  # categorical variable
                   y      = 'Symp',   # dependent variable
                   kind   = 'bar',    # kind of plot; options: bar, violin, box, boxen, strip, swarm, point
                   height = 4,        # size of the plot
                   )

### Defining colors

https://seaborn.pydata.org/tutorial/color_palettes.html \
https://matplotlib.org/stable/gallery/color/named_colors.html

In [ ]:
custom_palette = sns.choose_colorbrewer_palette('diverging') # options: sequential, qualitative
# custom_palette = sns.choose_light_palette()
# custom_palette = sns.choose_dark_palette()
# custom_palette = sns.choose_diverging_palette()

In [ ]:
sns.set_style("whitegrid") 
sns.set_context("paper", font_scale = 1.75) 

plot = sns.catplot(data    = data,  
                   x     = 'group', 
                   y       = 'Symp',    
                   kind    = 'bar',        
                   height  = 4, 
                   palette = custom_palette,
                   # palette = 'crest', # using a predifined color palette
                   # palette = {'mdd': 'tab:blue', 'hc': 'tab:green'}, # defining colors manually
                   )

### Finetuning

In [ ]:
sns.set_style("whitegrid") 
sns.set_context("paper", font_scale = 1.75) 

plot = sns.catplot(data      = data,  
                   x         = 'group', 
                   y         = 'Symp',    
                   kind      = 'bar',        
                   height    = 4,          
                   palette   = 'crest',                   
                   order     = ['HC', 'MDD'], # defining the order of conditions
                   estimator = np.mean,  # defining the statistical funcion to plot (e.g., np.median, np.var)
                   )

# Defining axis labels and title
plot.set(xlabel = 'Group', ylabel = 'Symptom Score')
plt.title('HC vs. MDD', weight = 'bold', fontsize = 20, y = 1.05)

# Saving the figure
#plot.savefig('plots/catplot.png')

### Adding a second categorical variable

In [ ]:
sns.set_style("whitegrid") 
sns.set_context("paper", font_scale = 1.75) 

plot = sns.catplot(data       = data,  
                   x          = 'group', 
                   y          = 'Symp',    
                   hue        = 'time', # additional categorical variable                   
                   kind       = 'bar',  
                   height     = 4,   
                   palette    = 'crest',                  
                   hue_order  = ['morning', 'evening'], # defining the order of conditions
                   legend_out = True,                   # defining position of legend               
                   )   

# Defining axis labels and title of legend
plot.set(xlabel = 'Group', ylabel = 'Symptom Score')
plot._legend.set_title('Time')     

### Adding a third categorical variable

In [ ]:
sns.set_style("whitegrid") 
sns.set_context("paper", font_scale = 1.75)

plot = sns.catplot(data      = data,  
                   x         = 'group', 
                   y         = 'Symp',    
                   hue       = 'time',  
                   col       = 'sex',        # third categorical variable
                   kind      = 'bar',        
                   height    = 4,          
                   palette   = 'crest',
                   col_order = ['Female', 'Male'], # defining the order of conditions
                   sharey    = False,      # shared y axis for both columns                 
                   )   

# Defining axis labels and title of legend
plot.set(xlabel = 'Group', ylabel = 'Symptom Score')
plot._legend.set_title('Time')     

# Defining title of columns and figure
plot.set_titles("{col_name}", size = 20)
plt.suptitle('Female vs. Male', x = 0.475, y = 1.1, weight = 'bold')  

plt.show()

## Different plot types for categorical data & Subplot structure

### Categorical estimate plots

https://seaborn.pydata.org/generated/seaborn.barplot.html \
https://seaborn.pydata.org/generated/seaborn.pointplot.html

In [ ]:
# Definition of figure properties
fig, sub_fig = plt.subplots(nrows   = 1,       # number of rows
                            ncols   = 2,       # number of columns (e.g., 2 plots next to each other)
                            figsize = (15, 6), # size of the figure (x/y)
                            sharey  = False)


# First plot
sns.barplot(ax      = sub_fig[0], # define position of the plot within the figure
            data    = data, 
            x       = 'group', 
            y       = 'Symp', 
            palette = 'crest')
sub_fig[0].set_title('Barplot', fontsize = 20)



# Second plot
sns.pointplot(ax   = sub_fig[1], # define position of the plot within the figure
              data = data, 
              x    = 'group', 
              y    = 'Symp')
sub_fig[1].set_title('Pointplot', fontsize = 20)



# Define axis labels for both subplots
for f in range(len(sub_fig)): 
    sub_fig[f].set_xlabel('Group')
    sub_fig[f].set_ylabel('Symptom Score') 

plt.tight_layout() # automatic optimal scaling
plt.show()  

### Categorical distribution plots

https://seaborn.pydata.org/generated/seaborn.boxplot.html \
https://seaborn.pydata.org/generated/seaborn.boxenplot.html \
https://seaborn.pydata.org/generated/seaborn.violinplot.html

In [ ]:
fig, sub_fig = plt.subplots(1, 3, figsize = (15, 5), sharey = True)


# First plot
sns.boxplot(ax = sub_fig[0], 
            data = data, 
            x = 'group', 
            y = 'Symp', 
            palette = 'crest')
sub_fig[0].set_title('Boxplot', fontsize = 20)


# Second plot
sns.boxenplot(ax = sub_fig[1], 
              data = data, 
              x = 'group', 
              y = 'Symp', 
              palette = 'crest')
sub_fig[1].set_title('Boxenplot', fontsize = 20)


# Third plot
sns.violinplot(ax = sub_fig[2], 
               data = data, 
               x = 'group', 
               y = 'Symp', 
               palette = 'crest')
sub_fig[2].set_title('Violinplot', fontsize = 20)


# Define axis labels for both subplots
for f in range(len(sub_fig)): 
    sub_fig[f].set_xlabel('Group')
    sub_fig[f].set_ylabel('Symptom Score')

plt.tight_layout()
plt.show()  

### Categorical scatterplots

https://seaborn.pydata.org/generated/seaborn.stripplot.html \
https://seaborn.pydata.org/generated/seaborn.swarmplot.html

In [ ]:
fig, sub_fig = plt.subplots(1, 2, figsize = (15, 6), sharey = False)


# First plot
sns.stripplot(ax = sub_fig[0], 
              data = data, 
              x = 'group', 
              y = 'Symp', 
              palette = 'crest')
sub_fig[0].set_title('Stripplot', fontsize = 20)


# Second plot
sns.swarmplot(ax = sub_fig[1], 
              data = data, 
              x = 'group', 
              y = 'Symp', 
              palette = 'crest')
sub_fig[1].set_title('Swarmplot', fontsize = 20)


# Define axis labels for both subplots
for f in range(len(sub_fig)): 
    sub_fig[f].set_xlabel('Group')
    sub_fig[f].set_ylabel('Symptom Score')
    
plt.tight_layout()
plt.show()              

### Combining plots

In [ ]:
plt.figure(figsize = (8, 5))

# First plot
sns.boxplot(data    = data, 
            x       = 'group', 
            y       = 'Symp', 
            palette = 'crest')

# Second plot
sns.stripplot(data  = data, 
              x     = 'group', 
              y     = 'Symp',
              color = 'black',
              jitter = 0.05)

plt.ylabel('Group')  
plt.xlabel('Symptom Score')
plt.show()

## Visualizing distributions of data 

### Histogram

https://seaborn.pydata.org/generated/seaborn.displot.html#seaborn.displot

In [ ]:
sns.set_style("white") 
sns.set_context("paper", font_scale = 1.75)

plot = sns.displot(data     = data, 
                   x        = "Emo", 
                   binwidth = 1)

plot.set(xlabel = 'Emotion Regulation Score')
plt.show()

In [ ]:
sns.set_style("white") 
sns.set_context("paper", font_scale = 1.75)

plot = sns.displot(data     = data, 
                   x        = 'Emo',
                   hue      = 'group',  
                   binwidth = 1)

plot.set(xlabel = 'Emotion Regulation Score')
plot._legend.set_title('Group') 
plt.show()

## Visualizing statistical relationships

### Scatter plot

https://seaborn.pydata.org/generated/seaborn.relplot.html

In [ ]:
sns.set_style("white") 
sns.set_context("paper", font_scale = 1.75)

plot = sns.relplot(data = data,
                   x    = 'Emo', 
                   y    = 'Symp', 
                   kind = 'scatter');

plot.figure.set_size_inches(8, 5)
plot.set(xlabel = 'Emotion Regulation Score', ylabel = 'Symptom Score')
plt.show()

### Scatterplot with linear regression model fit

https://seaborn.pydata.org/generated/seaborn.regplot.html

In [ ]:
sns.set_style("white") 
sns.set_context("paper", font_scale = 1.75)

plot = sns.regplot(data = data,
                   x    = "Emo", 
                   y    = "Symp")

plot.figure.set_size_inches(8, 5)
plot.set(xlabel = 'Emotion Regulation Score', ylabel = 'Symptom Score')
plt.show()

### Jointplot

https://seaborn.pydata.org/generated/seaborn.jointplot.html

In [ ]:
sns.set_style("white") 
sns.set_context("paper", font_scale = 1.75)

plot = sns.jointplot(data         = data,
                     x            = "Emo", 
                     y            = "Symp",
                     kind         = "reg", 
                     height       = 6,
                     marginal_kws = dict(bins = 20, fill = True))

plot.set_axis_labels('Emotion Regulation Score', 'Symptom Score', fontsize = 16)
plt.show()

In [ ]:
sns.set_style("white") 
sns.set_context("paper", font_scale = 1.75)

plot = sns.jointplot(data   = data,
                     x      = "Emo", 
                     y      = "Symp",
                     hue    = 'group',
                     kind   = "scatter",
                     height = 6)

plot.set_axis_labels('Emotion Regulation', 'Symptom Score', fontsize = 16)
plot.ax_joint.legend(title = 'Group')
plt.show()